In [555]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import os
import math
from tqdm import tqdm_notebook as tqdm
import warnings
warnings.filterwarnings('ignore')

In [97]:
DATA=pd.read_csv('For_matrix_2020_09_28.csv', sep = ',', encoding='utf-8')
DATA2=pd.read_csv('For_matrix_2020_09_29.csv', sep = ',', encoding='utf-8')
DATA2['time']=DATA2['time']+24*3600
DATA3=pd.read_csv('For_matrix_2020_09_30.csv', sep = ',', encoding='utf-8')
DATA3['time']=DATA3['time']+24*3600+24*3600
DATA=DATA.append(DATA2).append(DATA3)

In [98]:
DATA=DATA[['Дата и время поездки','Номер кристалла','TCAT','Код Маршрута','Код предприятия','mr_id','time','ssId','rl_id','rc_orderby']]

In [99]:
subway=pd.read_csv('stops_subway.csv', sep = ';', encoding='utf-8')

In [100]:
subway.head()

,ssId,Станция,ss_lat,ss_long
0,2081,Алабинская,53.209188,50.133092
1,2082,Российская,53.211449,50.149668
2,2083,Московская,53.203067,50.160567
3,2084,Гагаринская,53.200304,50.176584
4,2085,Спортивная,53.200954,50.199017


In [101]:
METRO=pd.read_csv('./METRO/2020.09.28.csv', sep=',', encoding='windows-1251')
METRO=METRO.merge(subway,on='Станция')
METRO['Код Маршрута']='subway'
METRO['mr_id']=0
METRO['rl_id']=0
METRO['rc_orderby']=0
METRO['time']=pd.to_datetime(METRO['Дата и время поездки'])
METRO['hour']=METRO['time'].dt.hour
METRO['min']=METRO['time'].dt.minute
METRO['sec']=METRO['time'].dt.second
METRO['time']=(METRO['hour'].astype(int))*3600+(METRO['min'].astype(int))*60+(METRO['sec'].astype(int))
METRO=METRO.drop('hour',axis=1).drop('sec',axis=1).drop('min',axis=1).drop('Терминал',axis=1).drop('NUM_TRIP',axis=1)
METRO=METRO[['Дата и время поездки','Номер кристалла','TCAT','Код Маршрута','Код предприятия','mr_id','time','ssId','rl_id','rc_orderby']]

In [102]:
METRO2=pd.read_csv('./METRO/2020.09.29.csv', sep=',', encoding='windows-1251')
METRO2=METRO2.merge(subway,on='Станция')
METRO2['Код Маршрута']='subway'
METRO2['mr_id']=0
METRO2['rl_id']=0
METRO2['rc_orderby']=0
METRO2['time']=pd.to_datetime(METRO2['Дата и время поездки'])
METRO2['hour']=METRO2['time'].dt.hour
METRO2['min']=METRO2['time'].dt.minute
METRO2['sec']=METRO2['time'].dt.second
METRO2['time']=(METRO2['hour'].astype(int))*3600+(METRO2['min'].astype(int))*60+(METRO2['sec'].astype(int))+24*3600
METRO2=METRO2.drop('hour',axis=1).drop('sec',axis=1).drop('min',axis=1).drop('Терминал',axis=1).drop('NUM_TRIP',axis=1)
METRO2=METRO2[['Дата и время поездки','Номер кристалла','TCAT','Код Маршрута','Код предприятия','mr_id','time','ssId','rl_id','rc_orderby']]

In [103]:
METRO3=pd.read_csv('./METRO/2020.09.30.csv', sep=',', encoding='windows-1251')
METRO3=METRO3.merge(subway,on='Станция')
METRO3['Код Маршрута']='subway'
METRO3['mr_id']=0
METRO3['rl_id']=0
METRO3['rc_orderby']=0
METRO3['time']=pd.to_datetime(METRO3['Дата и время поездки'])
METRO3['hour']=METRO3['time'].dt.hour
METRO3['min']=METRO3['time'].dt.minute
METRO3['sec']=METRO3['time'].dt.second
METRO3['time']=(METRO3['hour'].astype(int))*3600+(METRO3['min'].astype(int))*60+(METRO3['sec'].astype(int))+24*3600+24*3600
METRO3=METRO3.drop('hour',axis=1).drop('sec',axis=1).drop('min',axis=1).drop('Терминал',axis=1).drop('NUM_TRIP',axis=1)
METRO3=METRO3[['Дата и время поездки','Номер кристалла','TCAT','Код Маршрута','Код предприятия','mr_id','time','ssId','rl_id','rc_orderby']]

In [104]:
DATA=DATA.append(METRO).append(METRO2).append(METRO3)

In [105]:
del METRO
del METRO2
del METRO3
del DATA2
del DATA3

In [106]:
len(DATA)

1659660

привязали все валидации к сайтам

In [107]:
sites=pd.read_csv('stop_seq_v2.csv', sep = ';', encoding='utf-8')
sites=sites[['ssId','site_id']].drop_duplicates()
DATA=DATA.merge(sites,on='ssId', how='left')
len(DATA[DATA['ssId']==0])

166803

In [108]:
len(DATA)

1659660

ВЫБРАЛИ ДАННЫЕ ЗА ЧАС ПИК

In [109]:
#выбрала час пик только из перовго дня
DATA_pick=DATA[DATA['Дата и время поездки'].str.contains( '2020-09-28')] 
DATA_pick=DATA_pick[DATA_pick['Дата и время поездки'].str.contains( ' 07:')]\
    .append(DATA_pick[DATA_pick['Дата и время поездки'].str.contains( ' 08:')])\
    .append(DATA_pick[DATA_pick['Дата и время поездки'].str.contains( ' 09:')])

Убираем и запоминаем из выборки тех, у кого нет номера кристалла (невозможно точно привязать точку выхода)

In [110]:
DATA_bank_ssid=DATA_pick[DATA_pick['Номер кристалла'].isna()]

In [111]:
len(DATA_pick)

153888

In [112]:
len(DATA_bank_ssid)

48818

In [85]:
153888-39488

114400

ОСТАВЛЯЕМ В ЧАСЕ ПИК ТОЛЬКО ТЕХ, у кого есть номер кристалла:

In [18]:
DATA_pick=DATA_pick[~DATA_pick.index.isin(DATA_bank_ssid.index)]

In [86]:
len(DATA_pick)

105070

In [20]:
DATA_pick['time'].min()

25200

In [21]:
DATA_pick['time'].max()

35999

Выбрала из общего массива только те данные, где встречаются валидации из пикового промежутка:

In [22]:
DATA=DATA[DATA['Номер кристалла'].isin(DATA_pick['Номер кристалла'].unique())]

In [23]:
len(DATA)

439983

Выбрала только те значения, которые встречаются спустя 3 часа после начала часа пик

In [24]:
DATA=DATA[DATA['time']>DATA_pick['time'].min()+3*3600]

In [25]:
DATA=DATA.sort_values(['Номер кристалла','time','site_id'])

In [26]:
DATA_pick=DATA_pick.sort_values(['Номер кристалла','time','site_id'])

In [27]:
from tqdm import tqdm
k=0
res=DATA_pick
result = []
result2 = []
result3 = []
result4 = []
stops=pd.DataFrame()
with tqdm(total=DATA_pick.shape[0]) as pbar: 
    for index, row in DATA_pick.iterrows():
        pbar.update(1)
        stops=DATA[(DATA['Номер кристалла']==row['Номер кристалла'])
                    &(DATA['time']>(row['time']+3600*3))
                    &(DATA['site_id']!=row['site_id'])]
        if stops.empty:
            result.append(0)
            result2.append(0)
            result3.append(0)
            result4.append(0)
        else:
            stop_unic=stops['site_id'].values
            result.append(stop_unic[0])
            time_2=stops['time'].values
            result2.append(time_2[0])
            rl_id=stops['rl_id'].values
            result3.append(rl_id[0])
            rc_orderby=stops['rc_orderby'].values
            result4.append(rc_orderby[0])
res['site_id_2']=result
res['time_2']=result2
res['rl_id_2']=result3
res['rc_orderby_2']=result4

100%|████████████████████████████████████████████████████████████████████████| 105070/105070 [1:11:22<00:00, 24.54it/s]


In [30]:
res.to_csv('res_2020_09_28.csv')

In [432]:
del res

In [444]:
res=pd.read_csv('res_2020_09_28.csv')

320464/581500

In [445]:
len(res[(res['site_id_2']>0)&(res['site_id']>0)&(res['site_id_2']!=res['ssId'])])

77132

In [446]:
len(res)

105070

In [447]:
(77132)/(105070)

0.7341010754734939

In [454]:
res=res.fillna(0)

In [455]:
res['c']=1

In [456]:
res['c'].sum()

105070

In [457]:
res2=res.groupby(['site_id','site_id_2','mr_id']).agg({'c':np.nansum}).reset_index() 

Заполняем неприязавшиеся SsId наиболее подходящими остановками на маршруте

In [459]:
res_not_null=res[(res['site_id_2']>0)&(res['site_id']>0)] #выбрали из матрицы те строки, где все значения есть
res_null_last_stops=res[((res['site_id_2']==0)|(res['site_id_2'].isna()))
                        &(res['site_id']>0)].drop(['site_id_2'], axis=1) #выбрали только те строки, где есть остановка входа, но нет остановки выхода
res_null_first_stops=res[((res['site_id']==0)|(res['site_id'].isna()))
                        &(res['site_id_2']>0)].drop(['site_id'], axis=1) #выбрали только те строки, где есть остановка вЫхода, но нет остановки входа

Записываем всё, что не записалось и не определилось в массив res_null_null

In [460]:
res_null_null=res[~res.index.isin(res_not_null.index)]
res_null_null2=res_null_null[~res_null_null.index.isin(res_null_last_stops.index)]
del res_null_null
res_null_null=res_null_null2[~res_null_null2.index.isin(res_null_first_stops.index)].drop(['site_id'], axis=1).drop(['site_id_2'], axis=1)
del res_null_null2

In [461]:
res_null_null=res_null_null.reset_index()
res_null_last_stops=res_null_last_stops.reset_index()
res_null_first_stops=res_null_first_stops.reset_index()

In [462]:
len(res)-(len(res_not_null)+len(res_null_null)+len(res_null_last_stops)+len(res_null_first_stops))

0

In [463]:
len(DATA_bank_pick)

48818

In [464]:
res['c'].dropna().sum()

105070

In [465]:
res_null_null['c'].unique()

array([1], dtype=int64)

Добавляем банк

In [466]:
#добавили в выборку, где не привязались посл.остановки баноквские карты, где есть валидация начала
DATA_bank_ssid=DATA_bank_pick[DATA_bank_pick['site_id']>0]
DATA_bank_ssid['c']=1
DATA_bank_ssid=DATA_bank_ssid[['mr_id','site_id','c']]
res_null_last_stops=res_null_last_stops.append(DATA_bank_ssid).reset_index()

In [467]:
#добавили в выборку, где ничего не привязалось баноквские карты, где нет валидаций начала
DATA_bank_null=DATA_bank_pick[~DATA_bank_pick.index.isin(DATA_bank_ssid.index)]
DATA_bank_null['c']=1
DATA_bank_null=DATA_bank_null[['mr_id','site_id','c']]
res_null_null=res_null_null.append(DATA_bank_null).reset_index()

Сгруппировали все выгрузки, чтобы меньше хранить в оперативной памяти:

In [468]:
res_null_null=res_null_null.fillna(0)

In [469]:
res_null_null=res_null_null.groupby(['mr_id']).agg({'c':np.sum}).reset_index() 

In [470]:
res_null_last_stops=res_null_last_stops.groupby(['mr_id','site_id']).agg({'c':np.sum}).reset_index() 

In [471]:
res_null_first_stops=res_null_first_stops.groupby(['mr_id','site_id_2']).agg({'c':np.sum}).reset_index() 

In [472]:
res_not_null['c']=1
res_not_null=res_not_null.groupby(['mr_id','site_id','site_id_2']).agg({'c':np.sum}).reset_index() 

In [473]:
res_not_null=res_not_null.sort_values(by=['c'],ascending=False)

Определяем наиболее подходящие остановки для высадки на маршруте:

In [474]:
#Размазываем вход-выход для случая, когда мы знаем ВХОД
def input_func(res_not_null):
    input_=res_not_null.groupby(['site_id','site_id_2']).agg({'c':np.sum}).reset_index() 
    sum_mr_id=input_.groupby(['site_id']).agg({'c':np.sum}).reset_index() 
    input_=input_.merge(sum_mr_id, on =['site_id'])
    input_['c_x']=input_['c_x']/input_['c_y']
    input_=input_.drop(['c_y'], axis=1).rename(columns={'c_x':'c'})
    return input_

In [475]:
#Размазываем вход-выход для случая, когда мы знаем ВЫХОД
def exit_func(res_not_null):
    exit=res_not_null.groupby(['site_id','site_id_2']).agg({'c':np.sum}).reset_index() 
    sum_mr_id=exit.groupby(['site_id_2']).agg({'c':np.sum}).reset_index() 
    exit=exit.merge(sum_mr_id, on =['site_id_2'])
    exit['c_x']=exit['c_x']/exit['c_y']
    exit=exit.drop(['c_y'], axis=1).rename(columns={'c_x':'c'})
    return exit

In [476]:
#Там где не определился конец поездки, выбираем наиболее подходящую SsID для выхода (пропорционально выходу)
input_=input_func(res_not_null)
res_null_last_stops2=res_null_last_stops.merge(input_,on=['site_id'],how='left')
res_null_last_stops2['c']=res_null_last_stops2['c_x']*res_null_last_stops2['c_y']
res_null_last_stops2=res_null_last_stops2.groupby(['mr_id','site_id','site_id_2']).agg({'c':np.sum}).reset_index()

In [477]:
#записываем результат
res_null_last_stops2=res_null_last_stops2.fillna(0)
res_not_null=res_not_null.append(res_null_last_stops2[res_null_last_stops2['site_id_2']>0])

In [478]:
#записываем оставшиеся непривязавшиеся остановки
res_null_last_stops3=res_null_last_stops2[res_null_last_stops2['site_id_2']==0]
res_null_null=res_null_null.append(res_null_last_stops3.drop(['site_id_2'], axis=1).drop(['site_id'], axis=1))
del res_null_last_stops2
del res_null_last_stops

In [479]:
#Там где не определился конец поездки, выбираем наиболее подходящую SsID для выхода (пропорционально входу)
exit=exit_func(res_not_null)
res_null_first_stops2=res_null_first_stops.merge(exit,on=['site_id_2'],how='left')
res_null_first_stops2['c']=res_null_first_stops2['c_x']*res_null_first_stops2['c_y']
res_null_first_stops2=res_null_first_stops2.groupby(['mr_id','site_id','site_id_2']).agg({'c':np.sum}).reset_index()
res_null_first_stops2=res_null_firstt_stops2.fillna(0)
#записываем результат
res_not_null=res_not_null.append(res_null_first_stops2[res_null_first_stops2['site_id_2']>0])

In [480]:
#записываем оставшиеся непривязавшиеся остановки
res_null_first_stops3=res_null_first_stops2[res_null_first_stops2['site_id_2']==0]
res_null_null=res_null_null.append(res_null_first_stops3.drop(['site_id_2'], axis=1).drop(['site_id'], axis=1))
del res_null_first_stops2
del res_null_first_stops
del res_null_first_stops3

Снова группируем

In [481]:
res_not_null=res_not_null.groupby(['mr_id','site_id','site_id_2']).agg({'c':np.sum}).reset_index() 

In [482]:
res_not_null['c'].sum()

126443.0

In [483]:
#Размазываем вход-выход для случая, когда мы НИЧЕГО НЕ ЗНАЕМ
exit_input=res_not_null.groupby(['mr_id','site_id','site_id_2']).agg({'c':np.sum}).reset_index() 
sum_mr_id=exit_input.groupby(['mr_id']).agg({'c':np.sum}).reset_index() 
exit_input=exit_input.merge(sum_mr_id, on ='mr_id')
exit_input['c_x']=exit_input['c_x']/exit_input['c_y']
exit_input=exit_input.drop(['c_y'], axis=1).rename(columns={'c_x':'c'})

In [484]:
exit_input

,mr_id,site_id,site_id_2,c
0,0.0,31.0,1.0,0.000160
1,0.0,31.0,3.0,0.000017
2,0.0,31.0,4.0,0.000034
3,0.0,31.0,5.0,0.000034
4,0.0,31.0,7.0,0.000017
...,...,...,...,...
131709,306.0,794.0,376.0,0.011852
131710,306.0,794.0,377.0,0.011852
131711,306.0,794.0,507.0,0.002328
131712,306.0,794.0,577.0,0.002328


In [529]:
#Там где ничего не определилось, размазываем валидацию по всем парам вероятным :
res_null_null2=res_null_null.merge(exit_input,on=['mr_id'],how='left')

In [530]:
del res_not_null2

In [531]:
res_null_null2['c']=res_null_null2['c_x']*res_null_null2['c_y']

In [532]:
res_null_null2=res_null_null2.drop('c_x',axis=1).drop('c_y',axis=1)

In [533]:
res_not_null2=res_not_null.append(res_null_null2)

In [534]:
res_null_null2=res_null_null2.fillna(0)

In [535]:
res_not_null2=res_not_null2.groupby(['site_id','site_id_2']).agg({'c':np.sum}).reset_index()

In [536]:
itog=res_not_null2['c'].sum()

In [537]:
itog

137923.00000000003

In [538]:
DAY=pd.read_csv('For_matrix_2020_09_28.csv', sep = ',', encoding='utf-8')

Кол-во валидаций на НГПТ за день составило:

In [539]:
day=len(DAY)

In [540]:
day

581500

Кол-во валидаций на НГПТ за час пик день составило:

In [541]:
DAY_pick=DAY[DAY['Дата и время поездки'].str.contains( ' 07:')]\
    .append(DAY[DAY['Дата и время поездки'].str.contains( ' 08:')])\
    .append(DAY[DAY['Дата и время поездки'].str.contains( ' 09:')])
pick=len(DAY_pick)

In [542]:
pick

138074

Годовой поток по пригородным маршрутам составил 15369619

Переход к месяцу Октябрь от года - 10,73 (делим)

Переход к одному дню октября = 396 889 (средний будний день в октябре 2019 года)/10 852 053 (суммарный месячный поток в октябре 2019 за год)

Переход к часу - пик - делим кол-во валидаций за день к количеству валидаций за час пик 

In [543]:
(15369619/10.73)*(396889/10852053)*(pick/day)

12438.921399743787

In [544]:
res_not_null2['c']=res_not_null2['c']+(15369619/10.73)*(396889/10852053)*(pick/day)/itog

In [545]:
res_not_null2['c'].sum()

140292.2238797827

In [564]:
matrix=res_not_null2.groupby(['site_id','site_id_2']).agg({'c':np.sum}).reset_index()

In [565]:
matrix=matrix.sort_values(by='c',ascending=False)

In [566]:
matrix['site_id']=matrix['site_id'].astype('int64')

In [567]:
matrix['c']=round(matrix['c'])

In [570]:
matrix['c']=matrix['c'].astype(int)

In [572]:
matrix['site_id_2']=matrix['site_id_2'].astype('int64')

In [573]:
matrix

,site_id,site_id_2,c
15941,262,579,767
15951,262,995,503
17983,348,579,403
11870,201,579,392
15934,262,546,348
...,...,...,...
16228,292,86,1
16227,292,64,1
16226,292,33,1
25352,807,291,1


In [574]:
matrix.to_csv('primer_matrix.csv')

Считаем результаты помаршрутно

In [509]:
res_marsh=res_not_null2.groupby(['Код Маршрута','ssId','ssId_2']).agg({'c': np.sum}).reset_index()

KeyError: 'Код Маршрута'

In [510]:
res_marsh=res_marsh.merge(stops, left_on ='ssId',right_on ='ssId')\
                    .merge(stops, left_on ='ssId_2',right_on ='ssId')

NameError: name 'res_marsh' is not defined

In [ ]:
len(res_marsh)

In [ ]:
marsh=pd.read_csv('Маршруты.csv', sep='\t')

In [ ]:
marsh=marsh.rename(columns={'ID маршрута':'Код Маршрута'})

In [ ]:
res_marsh=res_marsh.merge(marsh, on ='Код Маршрута',how='left')

In [ ]:
res_marsh.to_csv('res.csv')

ПОДЕЛИТЬ НА 0,68 ВСЕ СТРОКИ!

ДОБАВИТЬ МЕТРО!!!!!

ПРОСТО ДОБАВИТЬ 15 369 619 в год (пересчитать сколько % это для часа пик)

Годовой поток по валидациям - 

In [ ]:
(15369619/10,73)*len(res)/len(DATA)

привязываем к сайтам

In [ ]:
sites.head()

In [ ]:
res.head()

In [ ]:
res2=res[['ssId','ssId_2']]
res2=res2.merge(sites,on='ssId')
res2=res2[['ssId','ssId_2','site_id']]
res2=res2.merge(sites,left_on='ssId_2', right_on='ssId')
res2['c']=1
res2=res2[['site_id_x','site_id_y','c']]
res2=res2.groupby(['site_id_x','site_id_y']).agg({'c':np.sum}).reset_index()
res2.to_csv('matrix_pick_2020_09_28.csv')

In [ ]:
stops=pd.read_csv('stops.csv', sep=',')

In [ ]:
res_marsh2=res_marsh2.merge(stops, left_on ='ssId',right_on ='ssId')\
                    .merge(stops, left_on ='ssId_2',right_on ='ssId')

In [ ]:
res_marsh2.head()

In [ ]:
from shapely.geometry import *
from shapely.ops import *
res_marsh2 = gpd.GeoDataFrame(res_marsh2, \
                              geometry = [LineString(xy) for xy in 
                              zip([Point(xy) for xy in zip(res_marsh2['ss_long_x'], res_marsh2['ss_lat_x'])], 
                              [Point(xy) for xy in zip(res_marsh2['ss_long_x'], res_marsh2['ss_lat_y'])])])

In [ ]:
res_marsh2.crs='epsg:4326'
res_marsh2.to_file("hour_pick.json", driver="GeoJSON", encoding='utf-8')

In [ ]:
res_marsh2.to_csv('res.csv')

In [ ]:

for_karta=res2.groupby(['Код Маршрута','Табельный номер','ssId','ss_lat','ss_long','start','finish']).agg({'c': np.sum}).reset_index()

In [ ]:
a_sta2 = gpd.GeoDataFrame(a_sta)
a_sta2.crs='epsg:4326'
a_sta2.to_file('XXXX.shp', driver='ESRI Shapefile', encoding='utf-8')